In [66]:
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
import torch

In [68]:
import numpy as np
import torchvision as tv
import pandas as pd
import numpy as np
import time

In [69]:
BATCH_SIZE=256

transforms = tv.transforms.Compose([
    tv.transforms.Resize(32),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=transforms, download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [70]:

train_dataset[0][0].shape

torch.Size([1, 32, 32])

In [71]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(1, 6, kernel_size=5, padding=0),
    torch.nn.Tanh(),
    torch.nn.AvgPool2d(2, stride=2),
    torch.nn.Conv2d(6, 16, kernel_size=5),
    torch.nn.Tanh(),
    torch.nn.AvgPool2d(2, stride=2),
    torch.nn.Conv2d(16, 120, kernel_size=5),
    torch.nn.Flatten(),
    torch.nn.Linear(120, 84),
    torch.nn.Tanh(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(84, 10)
)

In [72]:
'''model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, 10)
)'''

'model = torch.nn.Sequential(\n    torch.nn.Flatten(),\n    torch.nn.Linear(784, 1024),\n    torch.nn.BatchNorm1d(1024),\n    torch.nn.ReLU(),\n    torch.nn.Dropout(0.1),\n    torch.nn.Linear(1024, 512),\n    torch.nn.BatchNorm1d(512),\n    torch.nn.ReLU(),\n    torch.nn.Dropout(0.2),\n    torch.nn.Linear(512, 256),\n    torch.nn.BatchNorm1d(256),\n    torch.nn.ReLU(),\n    torch.nn.Dropout(0.3),\n    torch.nn.Linear(256, 10)\n)'

In [73]:
model

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): Tanh()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Tanh()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=120, out_features=84, bias=True)
  (9): Tanh()
  (10): Dropout(p=0.1, inplace=False)
  (11): Linear(in_features=84, out_features=10, bias=True)
)

In [74]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=.01)
num_epochs = 150

In [75]:
#from prettytable import PrettyTable
#x = PrettyTable()

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [76]:
#x.field_names = ["EPOCH", "TimeTaked", "train_loss", "train_acc", "test_loss", "test_acc"]
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0.
        start = time.time()

        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        test_iters, test_passed = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
        count_ep = ep+1
        if count_ep % 15 == 0:
            print("epoch: {}, taked_time: {:.3f}, train_loss: {:.3f}, train_acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}".format(
                count_ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
                test_loss / test_iters, test_acc / test_passed)
            )
        #x.add_row([round(ep,3),round(time.time() - start,3), round(train_loss / train_iters,3), round(train_acc / train_passed,3), round(test_loss / test_iters,3), round(test_acc / test_passed,3)])
        # print(x.get_string())


Using cpu device


In [78]:
train_model()

epoch: 15, taked_time: 41.833, train_loss: 0.599, train_acc: 0.782, test_loss: 0.563, test_acc: 0.789


KeyboardInterrupt: 